# Hotel price dataset

- train 데이터의 구매정보 : clickout item 일때 impressions 의 price
- item_metadata 의 호텔정보 : 메타데이터에 있는 호텔만 매칭
- timestamp 고려못했음

In [1]:
import pandas as pd 
import numpy as np

In [2]:
train = pd.read_csv('../data/train.csv', sep = ',', encoding = 'utf-8')
item_metadata = pd.read_csv('../data/item_metadata.csv', sep = ',', encoding = 'utf-8')

In [3]:
print('train:', train.shape)
print('item_metadata:', item_metadata.shape)

train: (15932992, 12)
item_metadata: (927142, 2)


## train data 에서 hotel 정보

In [4]:
train.head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,008UN2RY0DMY,cd10940538005,1541031243,1,search for destination,"Rochester, USA",US,"Rochester, USA",desktop,NaN,NaN,NaN
1,008UN2RY0DMY,cd10940538005,1541031463,2,change of sort order,interaction sort button,US,"Rochester, USA",desktop,NaN,NaN,NaN
2,008UN2RY0DMY,cd10940538005,1541031473,3,change of sort order,distance and recommended,US,"Rochester, USA",desktop,NaN,NaN,NaN


In [5]:
train.groupby('action_type').size()

action_type
change of sort order        400584
clickout item              1586586
filter selection            695917
interaction item deals      411040
interaction item image    11860750
interaction item info       285402
search for destination      403066
search for item             152203
search for poi              137444
dtype: int64

### clickout_only

In [6]:
clickout_only = train.loc[train['action_type'] == 'clickout item']
clickout_only = clickout_only.loc[:, ['impressions', 'prices']]
clickout_only.head()

,impressions,prices
29,7818446|51315|2133708|119638|86359|133581|6688...,163|75|93|98|169|201|129|117|108|170|133|80|87...
32,6010044|5433188|5156000|4780400|5137162|992819...,38|35|54|37|73|37|37|30|71|75|64|26|25|37|40|3...
39,3132957|100226|1954167|9462680|2776177|929533|...,25|324|67|52|57|21|90|240|64|37|70|60|167|45|6...
73,9112592|102414|9140448|5654906|4341120|9739716...,263|114|263|273|122|194|236|161|219|113|110|19...
393,3505518|1834297|4837106|3990396|2042495|486747...,47|50|45|40|45|40|40|50|50|45|50|45|38|49|48|3...


### hotel_id , mean_price, cnt_clickout dataframe 생성하기

In [7]:
impressions_split = clickout_only.impressions.str.split('|')
prices_split = clickout_only.prices.str.split('|')

### <참고> split 된 자료형

In [11]:
len(impressions_split), len(prices_split)

(1586586, 1586586)

In [9]:
print(impressions_split.head())
print(impressions_split.tail())
#hotel_contain_dic['{0}'.format(impressions_split[0])]

29     [7818446, 51315, 2133708, 119638, 86359, 13358...
32     [6010044, 5433188, 5156000, 4780400, 5137162, ...
39     [3132957, 100226, 1954167, 9462680, 2776177, 9...
73     [9112592, 102414, 9140448, 5654906, 4341120, 9...
393    [3505518, 1834297, 4837106, 3990396, 2042495, ...
Name: impressions, dtype: object
15932786    [2788164, 41702, 3533528, 914143, 8909434, 137...
15932787    [5510936, 1476545, 3251274, 2124270, 4779238, ...
15932830    [3201672, 9554096, 9303240, 2847178, 2635628, ...
15932832    [4437776, 5801200, 95357, 4915330, 93282, 4111...
15932991    [2813114, 1291318, 11142246, 477981, 2784918, ...
Name: impressions, dtype: object


In [10]:
print(prices_split.head())
print(prices_split.tail())

29     [163, 75, 93, 98, 169, 201, 129, 117, 108, 170...
32     [38, 35, 54, 37, 73, 37, 37, 30, 71, 75, 64, 2...
39     [25, 324, 67, 52, 57, 21, 90, 240, 64, 37, 70,...
73     [263, 114, 263, 273, 122, 194, 236, 161, 219, ...
393    [47, 50, 45, 40, 45, 40, 40, 50, 50, 45, 50, 4...
Name: prices, dtype: object
15932786    [61, 47, 116, 57, 49, 65, 67, 157, 78, 78, 94,...
15932787    [115, 50, 36, 173, 22, 49, 57, 60, 31, 38, 13,...
15932830    [49, 44, 13, 24, 21, 30, 25, 13, 25, 33, 37, 3...
15932832    [34, 51, 74, 31, 68, 78, 46, 24, 8, 57, 164, 5...
15932991    [108, 77, 185, 156, 168, 86, 112, 224, 161, 12...
Name: prices, dtype: object


## hotel_contain_dic 딕셔너리 만들기
## hotel_not_contain_dic
- key : item_metadata 의 item_id
- value : train data 의 clickout 시 나타나는 price list

In [12]:
# item_metadata 에 있는 호텔 리스트의 가격정보 담기
hotel_contain_dic = {}

for i in range(0, len(item_metadata['item_id'])):
    
    hotel_contain_dic['{0}'.format(item_metadata['item_id'][i])] = []

    
# 구매정보가 있지만 item_metadata에 없는 호텔리스트
hotel_not_contain_list = []

# metadataset 안에 있는 것과 매핑
for i in range(0, len(impressions_split)):
    
    for x, y in zip(impressions_split.iloc[i], prices_split.iloc[i]):
        
        try:
            hotel_contain_dic['{0}'.format(x)].append(int(y))
            break
            
        except KeyError:
            print('{0} is not in our keys'.format(x))
            hotel_not_contain_list.append(x)
            
            

8952764 is not in our keys
8299034 is not in our keys
10719104 is not in our keys
5736924 is not in our keys
6916084 is not in our keys
5814172 is not in our keys
8474370 is not in our keys
5814172 is not in our keys
8721352 is not in our keys
8969600 is not in our keys
9927784 is not in our keys
8530600 is not in our keys
11062878 is not in our keys
6473528 is not in our keys
9050092 is not in our keys
9400108 is not in our keys
8994580 is not in our keys
10130312 is not in our keys
11097532 is not in our keys
2218064 is not in our keys
8454054 is not in our keys
10889710 is not in our keys
5432214 is not in our keys
5427634 is not in our keys
8672908 is not in our keys
9301238 is not in our keys
10874050 is not in our keys
8985840 is not in our keys
10941440 is not in our keys
7122966 is not in our keys
10889710 is not in our keys
10585958 is not in our keys
7346452 is not in our keys
8947714 is not in our keys
10675072 is not in our keys
10888766 is not in our keys
10888766 is not i

8721352 is not in our keys
11007952 is not in our keys
11007952 is not in our keys
9535008 is not in our keys
9535008 is not in our keys
6859080 is not in our keys
8238272 is not in our keys
9500574 is not in our keys
1738367 is not in our keys
9042238 is not in our keys
11106666 is not in our keys
10889710 is not in our keys
7309412 is not in our keys
10889710 is not in our keys
6473528 is not in our keys
9520364 is not in our keys
9484070 is not in our keys
7828880 is not in our keys
8220214 is not in our keys
4466176 is not in our keys
11101014 is not in our keys
7129272 is not in our keys
9021038 is not in our keys
10719104 is not in our keys
5219948 is not in our keys
10646254 is not in our keys
5589536 is not in our keys
6473528 is not in our keys
8598328 is not in our keys
4466176 is not in our keys
8985840 is not in our keys
5405700 is not in our keys
10889710 is not in our keys
9400036 is not in our keys
5405700 is not in our keys
6942612 is not in our keys
9901680 is not in o

10026120 is not in our keys
10026120 is not in our keys
10238094 is not in our keys
8706968 is not in our keys
10556358 is not in our keys
10889710 is not in our keys
10889710 is not in our keys
5782144 is not in our keys
9021038 is not in our keys
6473528 is not in our keys
5486622 is not in our keys
5612126 is not in our keys
10599618 is not in our keys
11151062 is not in our keys
5814176 is not in our keys
5595564 is not in our keys
5814072 is not in our keys
11062878 is not in our keys
5777918 is not in our keys
8552068 is not in our keys
8598328 is not in our keys
5750368 is not in our keys
10889710 is not in our keys
6916084 is not in our keys
4757300 is not in our keys
4466176 is not in our keys
8501344 is not in our keys
8969600 is not in our keys
6953208 is not in our keys
11103906 is not in our keys
6473528 is not in our keys
5777682 is not in our keys
5589536 is not in our keys
8576928 is not in our keys
10889710 is not in our keys
5808934 is not in our keys
10889710 is not 

In [13]:
print('item_metadata:', item_metadata.shape)
print('hotel_contain_dic:', len(hotel_contain_dic))
print('hotel_not_contain_list: ', len(hotel_not_contain_list))

item_metadata: (927142, 2)
hotel_contain_dic: 927142
hotel_not_contain_list:  811


In [14]:
hotel_contain_dic

{'5121': [93,
  105,
  238,
  238,
  119,
  109,
  99,
  149,
  149,
  119,
  171,
  168,
  199,
  105,
  111,
  109],
 '5555': [346,
  258,
  218,
  218,
  295,
  295,
  259,
  209,
  270,
  210,
  213,
  229,
  239,
  1486,
  304,
  231,
  263,
  285,
  426,
  213,
  229,
  262,
  262,
  267,
  259,
  229,
  270,
  222,
  246,
  205,
  1277,
  284,
  268,
  1305,
  1159,
  251,
  373,
  300,
  423,
  218,
  279,
  279,
  303,
  290,
  241,
  1407,
  223,
  286],
 '5684': [90, 100, 107, 84, 130, 580],
 '5777': [88, 73, 80, 107, 111, 185, 73, 87],
 '6044': [70, 79, 78, 78, 67, 100, 105, 105, 58, 79, 84],
 '6095': [],
 '6182': [],
 '6257': [72],
 '8022': [97, 154],
 '8087': [424, 184, 254, 201, 219, 239, 273, 219],
 '8450': [102, 199, 233, 181, 90, 188],
 '9331': [39, 51, 39, 54, 58, 45],
 '11601': [1200],
 '11646': [103, 89, 87, 108, 105, 61, 117],
 '11886': [121, 87, 97, 102, 102, 90, 82],
 '11892': [128, 124, 151, 151, 131, 203, 125, 116, 91, 131, 120, 124, 112],
 '12006': [80, 125, 

## hotel_information 딕셔너리
- item_id 
- mean_price 
- min_price
- max_price
- cnt_clickout : 리스트 크기

In [15]:
hotel_information = {}

item_id_list = []
mean_price_list = []
# min_price_list = []
# max_price_list = []
cnt_clickout_list = []

for i in hotel_contain_dic:
    
    # 계산하기
    prices = np.array(hotel_contain_dic.get(i))
    mean_price = float(np.mean(prices))
#     min_price = np.min(prices)
#     max_price = np.max(prices)
    cnt_clickout = len(prices)
    
    
    # list에 담기
    item_id_list.append(i)
    mean_price_list.append(mean_price)
#     min_price_list.append(min_price)
#     max_price_list.append(max_price)
#     std
    cnt_clickout_list.append(cnt_clickout)
    

# 딕셔너리 만들기
hotel_information['item_id'] = item_id_list
hotel_information['mean_price'] = mean_price_list
# hotel_information['min_price'] = min_price_list
# hotel_information['max_price'] = max_price_list
hotel_information['cnt_clickout'] = cnt_clickout_list


C:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## dictionary to dataframe
#### hotel_information_dic 

In [16]:
# key 확인
hotel_information.keys()
# vlaue 확인
#hotel_information.values()

dict_keys(['item_id', 'mean_price', 'cnt_clickout'])

In [17]:
hotel_information

{'item_id': ['5121',
  '5555',
  '5684',
  '5777',
  '6044',
  '6095',
  '6182',
  '6257',
  '8022',
  '8087',
  '8450',
  '9331',
  '11601',
  '11646',
  '11886',
  '11892',
  '12006',
  '12016',
  '12125',
  '12436',
  '12921',
  '14747',
  '15255',
  '15662',
  '15686',
  '16060',
  '16311',
  '17111',
  '17233',
  '17966',
  '18143',
  '19197',
  '19419',
  '19492',
  '19602',
  '19800',
  '19922',
  '20648',
  '20854',
  '22684',
  '23627',
  '23741',
  '23849',
  '23882',
  '25017',
  '25169',
  '25854',
  '26812',
  '28036',
  '28628',
  '28719',
  '28926',
  '29230',
  '31048',
  '31918',
  '32596',
  '32833',
  '32852',
  '32959',
  '34049',
  '34057',
  '34326',
  '34558',
  '34641',
  '34740',
  '35437',
  '36209',
  '36692',
  '37271',
  '38084',
  '38859',
  '39086',
  '39114',
  '39636',
  '39681',
  '39718',
  '39834',
  '39924',
  '39961',
  '40034',
  '40894',
  '41094',
  '41138',
  '41226',
  '41556',
  '41878',
  '41975',
  '42191',
  '42496',
  '42571',
  '42639',


## df_hotel_price dataframe

- item metadata 랑 mapping

In [18]:
# 데이터프레임
df_hotel_price = pd.DataFrame(hotel_information)
df_hotel_price.head(30)

,item_id,mean_price,cnt_clickout
0,5121,142.562500,16
1,5555,376.750000,48
2,5684,181.833333,6
3,5777,100.500000,8
4,6044,82.090909,11
5,6095,NaN,0
6,6182,NaN,0
7,6257,72.000000,1
8,8022,125.500000,2
9,8087,251.625000,8


In [19]:
# 저장
df_hotel_price.to_csv('../data/df_hotel_price.csv', index = False)






### <연습> np

In [38]:
a = np.array(hotel_contain_dic.get('5121'))
#hotel_contain_dic.get('5121')
a

array([ 93, 105, 238, 238, 119, 109,  99, 149, 149, 119, 171, 168, 199,
       105, 111, 109])

In [52]:
float(np.mean(a))

142.5625

In [40]:
len(a)

16

In [82]:
k = []
#k.append(4)
k

[]

In [83]:
np.min(k)
np.max(k)

ValueError: zero-size array to reduction operation minimum which has no identity

### <참고> 포함안된거 처리해보다 실패!

In [18]:
# hotel_not_contain_dic = {}
# for i in range(0, len(hotel_not_contain_list)):
#     hotel_not_contain_dic['{0}'.format(hotel_not_contain_list[i])] = []
# hotel_not_contain_dic

In [19]:
# hotel_not_contain_dic

In [20]:
#for i in range(0, len(impressions_split)):
#    for x, y in zip(impressions_split.iloc[i], prices_split.iloc[i]):
#        try:
#            hotel_not_contain_dic['{0}'.format(x)].append(y)
#            break
#        except KeyError:
#            print('{0} is not in our keys'.format(x))

### <참고> split 된 자료형

In [76]:
impressions_split.tail()

15932786    [2788164, 41702, 3533528, 914143, 8909434, 137...
15932787    [5510936, 1476545, 3251274, 2124270, 4779238, ...
15932830    [3201672, 9554096, 9303240, 2847178, 2635628, ...
15932832    [4437776, 5801200, 95357, 4915330, 93282, 4111...
15932991    [2813114, 1291318, 11142246, 477981, 2784918, ...
Name: impressions, dtype: object

In [78]:
prices_split.head()

29     [163, 75, 93, 98, 169, 201, 129, 117, 108, 170...
32     [38, 35, 54, 37, 73, 37, 37, 30, 71, 75, 64, 2...
39     [25, 324, 67, 52, 57, 21, 90, 240, 64, 37, 70,...
73     [263, 114, 263, 273, 122, 194, 236, 161, 219, ...
393    [47, 50, 45, 40, 45, 40, 40, 50, 50, 45, 50, 4...
Name: prices, dtype: object

In [79]:
prices_split.tail()

15932786    [61, 47, 116, 57, 49, 65, 67, 157, 78, 78, 94,...
15932787    [115, 50, 36, 173, 22, 49, 57, 60, 31, 38, 13,...
15932830    [49, 44, 13, 24, 21, 30, 25, 13, 25, 33, 37, 3...
15932832    [34, 51, 74, 31, 68, 78, 46, 24, 8, 57, 164, 5...
15932991    [108, 77, 185, 156, 168, 86, 112, 224, 161, 12...
Name: prices, dtype: object

### <연습> zip

In [1]:
# for x, y in zip(impressions_split.iloc[0], prices_split.iloc[0]):
#     print(x, y)
#     print(type(x))
#     print(type(y))

### <연습> dic

In [55]:
p = [3132957, 100226, 1954167, 9462680, 2776177]
pp = [25, 324, 67, 52, 57]
h_dic = {}
for i in range(0, len(p)):
    h_dic['{0}'.format(p[i])] = set()

for i in range(0, len(pp)):
    h_dic['{0}'.format(p[i])].add(pp[i])
    
#    for j in pp:
#        h_dic['{0}'.format(i)].add(j) 

In [127]:
h_dic

{'3132957': {25},
 '100226': {324},
 '1954167': {67},
 '9462680': {52},
 '2776177': {1, 57}}

In [144]:
type(h_dic.get('2776177'))

set

In [158]:
a = []
a.append('1')
a.append(1)
a

['1', 1]

In [ ]:
impressions_list = []
prices_list = []
for i in clickout_only.index:
    for x, y in zip(impressions_split, prices_split):
        #price_set = set()
        #print(x, y) 
        #print(" ") 
        for i in range(0, len(x)):
            impressions_list.append(x[i])
            prices_list.append(int(y[i]))

In [ ]:
len(impressions_list), len(prices_list)

In [ ]:
dd = {}
for i, j in zip(impressions_list, prices_list):
    dd.

In [48]:
print(a.head())
print(a.tail())

      hotel  price
0   1001753     37
1   1001785     25
2    100226    324
3  10055274     75
4   1017368    181
       hotel  price
118  9659264     23
119  9724982    175
120  9734016     84
121  9739716    194
122  9928194     37


In [44]:
#a = pd.DataFrame({'hotel': impressions_list, 
#             'price': prices_list}).groupby('hotel').mean().reset_index()
print(a.head())
print(a.tail())

      hotel  price
0   1001753     37
1   1001785     25
2    100226    324
3  10055274     75
4   1017368    181
       hotel  price
118  9659264     23
119  9724982    175
120  9734016     84
121  9739716    194
122  9928194     37


In [64]:
item_metadata['item_id'][0]

5121

### <참고> 원래 hotel_contain_dic

In [15]:
# key : item_metadata 의 item_id
# value : train data 의 clickout 시 나타나는 price list
hotel_contain_dic

{'5121': [],
 '5555': [],
 '5684': [],
 '5777': [],
 '6044': [],
 '6095': [],
 '6182': [],
 '6257': [],
 '8022': [],
 '8087': [],
 '8450': [],
 '9331': [],
 '11601': [],
 '11646': [],
 '11886': [],
 '11892': [],
 '12006': [],
 '12016': [],
 '12125': [],
 '12436': [],
 '12921': [],
 '14747': [],
 '15255': [],
 '15662': [],
 '15686': [],
 '16060': [],
 '16311': [],
 '17111': [],
 '17233': [],
 '17966': [],
 '18143': [],
 '19197': [],
 '19419': [],
 '19492': [],
 '19602': [],
 '19800': [],
 '19922': [],
 '20648': [],
 '20854': [],
 '22684': [],
 '23627': [],
 '23741': [],
 '23849': [],
 '23882': [],
 '25017': [],
 '25169': [],
 '25854': [],
 '26812': [],
 '28036': [],
 '28628': [],
 '28719': [],
 '28926': [],
 '29230': [],
 '31048': [],
 '31918': [],
 '32596': [],
 '32833': [],
 '32852': [],
 '32959': [],
 '34049': [],
 '34057': [],
 '34326': [],
 '34558': [],
 '34641': [],
 '34740': [],
 '35437': [],
 '36209': [],
 '36692': [],
 '37271': [],
 '38084': [],
 '38859': [],
 '39086': [],
 '39